In [1]:
save_file1 = c3.HindcastFile.get('007e2816-3905-434b-b957-f7b0d0b03f46')
save_file2 = c3.HindcastFile.get('007e88f7-b86d-4379-a194-a73e2d003afb')

In [3]:
# hycom_file = c3.HycomUtil.nc_open(save_file.file.url)
# # save_file.file

In [4]:
# from datetime import datetime, timedelta
# # hycom_file['time']

# time_origin = datetime.strptime(hycom_file.variables['time'].__dict__['time_origin'] + ' +0000',
#                                         '%Y-%m-%d %H:%M:%S %z')

In [16]:
#test the time performance, copied from process() below

import time as tm
from datetime import datetime, timedelta
start_time = tm.time()


this = c3.HindcastFile.get("007e2816-3905-434b-b957-f7b0d0b03f46")
hycom_file = c3.HycomUtil.nc_open(this.file.url)
time_origin = datetime.strptime(hycom_file.variables['time'].__dict__['time_origin'] + ' +0000',
                                        '%Y-%m-%d %H:%M:%S %z')
latitudes = range(len(hycom_file['lat']))
longitudes = range(len(hycom_file['lon']))
time = time_origin + timedelta(hours=hycom_file.variables['time'][:][0])



print("begin processing time: ", tm.time() - start_time)
lst = []
times = []
for i in latitudes[:10]:
    for j in longitudes[:10]:
        start = tm.time()
        data = c3.SurfaceHindcastData( # look at the code to understand, TimeDataPoint (Check out Type)
            **{
                'start': time,
                'parent': 'HNDCST_SRFC_' + str(i) + '-' + str(j),
                'name': 'water_u',  # variable
                'for': time,           # timestamp
                'water_u': hycom_file.variables['water_u'][:].data[0,0,i,j],
                'water_v': hycom_file.variables['water_v'][:].data[0,0,i,j]
            }
        )
        lst.append(data)
        
        times.append(tm.time() - start)

print(sum(times) / len(times)) #came out to 0.3022856497764587

begin processing time:  0.8753042221069336
0.3022856497764587


In [13]:
#beginning processing: varies, got 8 seconds once, 0.8 seconds once

#from above: ~30 seconds for every 100 list items, ~80 seconds per latitude (276 longitudes)
print(len(latitudes), len(longitudes))
print("hours", 159*80 / 3600)

159 276
hours 3.533333333333333


In [1]:
# Prototype for prosessing hindcast files
from datetime import datetime, timedelta

def process(this):
    """ Process a single Hindcast NetCDF file into the Hindcast__Data types"""
    # extract surface data for a variable
    hycom_file = c3.HycomUtil.nc_open(this.file.url)
    time_origin = datetime.strptime(hycom_file.variables['time'].__dict__['time_origin'] + ' +0000',
                                        '%Y-%m-%d %H:%M:%S %z')
    
    # extract lat-long, or derive this from types
    # Not yet done:determine the offset for each based on the subsetOptions for this file
    # Note: for now it's just an integer list
    latitudes = range(len(hycom_file['lat']))
    longitudes = range(len(hycom_file['lon']))
    time = time_origin + timedelta(hours=hycom_file.variables['time'][:][0])

    # Create list of instantiated SurfaceHindcastData types
    # Create a parent id for each lat-long pair in the file.
    # use lat-long indicies to create a string parent id

    data_records = [
        c3.SurfaceHindcastData( # look at the code to understand, TimeDataPoint (Check out Type)
            **{
                'start': time,
                'parent': 'HNDCST_SRFC_' + str(i) + '-' + str(j),
                'name': 'water_u',  # variable
                'for': time,           # timestamp
                'water_u': hycom_file.variables['water_u'][:].data[0,0,i,j],
                'water_v': hycom_file.variables['water_v'][:].data[0,0,i,j]
            }
        )
        for i in latitudes[:1]
            for j in longitudes[:1]
    ]
    # upsert to data store
    c3.SurfaceHindcastData.upsertBatch(data_records)

    # close the file ds, url
    c3.HycomUtil.nc_close(ds=hycom_file, url=this.file.url)
    
    # aggregation in space is possible but a bit more complex

In [8]:
process(c3.HindcastFile.get('007e2816-3905-434b-b957-f7b0d0b03f46'))
process(c3.HindcastFile.get('007e88f7-b86d-4379-a194-a73e2d003afb'))

NameError: name 'process' is not defined

In [ ]:
data_records = [
        c3.SurfaceHindcastDataSeries(
            **{ 'id': 'HNDCST_SRFC_0-0',
                'latLongPair': 'GOMu0.04_1-1',
                
            }
        )
    ]

In [7]:
c3.SurfaceHindcastDataSeries.upsertBatch(data_records)

c3.ObjList<SurfaceHindcastDataSeries>(
 stats=c3.Stats(
         createdObjCount=1,
         updatedObjCount=0,
         removedObjCount=0,
         failedObjCount=0,
         objCount=1,
         dbWait=0.0,
         time=0.181,
         dbTime=0.013696359,
         objsPerSecond=5.524861878453039,
         dbObjsPerSecond=73.012104895907),
 objs=c3.Arry<SurfaceHindcastDataSeries>([c3.SurfaceHindcastDataSeries(
         id='HNDCST_SRFC_1-1',
         meta=c3.Meta(
                created=datetime.datetime(2021, 10, 15, 23, 48, 19, tzinfo=datetime.timezone.utc),
                updated=datetime.datetime(2021, 10, 15, 23, 48, 19, tzinfo=datetime.timezone.utc),
                timestamp=datetime.datetime(2021, 10, 15, 23, 48, 19, tzinfo=datetime.timezone.utc)),
         version=1)]))

In [62]:
objs_list = c3.HycomLatLongPair.fetch(spec={"include": "id", "filter": "lat > 18 && lat < 19 && lon > -98 && lon < -97", "order": "j,i"}).objs

ids_list = [obj.id for obj in objs_list]

['GOMu0.04_1-0',
 'GOMu0.04_2-0',
 'GOMu0.04_3-0',
 'GOMu0.04_4-0',
 'GOMu0.04_5-0',
 'GOMu0.04_6-0',
 'GOMu0.04_7-0',
 'GOMu0.04_8-0',
 'GOMu0.04_9-0',
 'GOMu0.04_10-0',
 'GOMu0.04_11-0',
 'GOMu0.04_12-0',
 'GOMu0.04_13-0',
 'GOMu0.04_14-0',
 'GOMu0.04_15-0',
 'GOMu0.04_16-0',
 'GOMu0.04_17-0',
 'GOMu0.04_18-0',
 'GOMu0.04_19-0',
 'GOMu0.04_20-0',
 'GOMu0.04_21-0',
 'GOMu0.04_22-0',
 'GOMu0.04_23-0',
 'GOMu0.04_24-0',
 'GOMu0.04_1-1',
 'GOMu0.04_2-1',
 'GOMu0.04_3-1',
 'GOMu0.04_4-1',
 'GOMu0.04_5-1',
 'GOMu0.04_6-1',
 'GOMu0.04_7-1',
 'GOMu0.04_8-1',
 'GOMu0.04_9-1',
 'GOMu0.04_10-1',
 'GOMu0.04_11-1',
 'GOMu0.04_12-1',
 'GOMu0.04_13-1',
 'GOMu0.04_14-1',
 'GOMu0.04_15-1',
 'GOMu0.04_16-1',
 'GOMu0.04_17-1',
 'GOMu0.04_18-1',
 'GOMu0.04_19-1',
 'GOMu0.04_20-1',
 'GOMu0.04_21-1',
 'GOMu0.04_22-1',
 'GOMu0.04_23-1',
 'GOMu0.04_24-1',
 'GOMu0.04_1-2',
 'GOMu0.04_2-2',
 'GOMu0.04_3-2',
 'GOMu0.04_4-2',
 'GOMu0.04_5-2',
 'GOMu0.04_6-2',
 'GOMu0.04_7-2',
 'GOMu0.04_8-2',
 'GOMu0.04_9-2',
 

In [74]:
# Define simple metric
my_metric = c3.SimpleMetric(id = "AverageWaterU_HycomLatLongPair",
                            name = "AverageWaterU",
                            description = "Calculates average surface X-velocity for the given interval",
                            srcType = "HycomLatLongPair",
                            path = "surfaceHindcastData",
                            expression = "avg(avg(normalized.data.water_u))"
                           )

my_metric.toJson()
# Create EvalMetricsSpec
my_spec = c3.EvalMetricsSpec(
            ids = ['GOMu0.04_1-0'],
            expressions = ["TestAverageWaterU"], # does this refer to the one above?
            start = "2021-09-01",
            end = "2021-09-30",
            interval = "DAY", )


In [75]:
evalMetricsResult = c3.HycomLatLongPair.evalMetrics(spec=my_spec) #eventually swap for just evalMetrics function (less metadata)
evalMetricsResult

500 - NotClassified - c3.love.exceptions.C3RuntimeException_wrapIt [8363.1266]
message: "wrapped RuntimeException: Field 'i' does not exist in expression: '[latLongPair, ascending(i), ascending(j)]' for type 'SurfaceHindcastDataSeries'."
JSON: {"spec": {"type": "EvalMetricsSpec", "start": "2021-09-01T00:00:00", "end": "2021-09-30T00:00:00", "timeZone": "NONE", "interval": "DAY", "cache": false, "ids": ["GOMu0.04_1-0"], "expressions": ["TestAverageWaterU"], "continueOnError": false}}


C3RuntimeException: 500 - NotClassified - c3.love.exceptions.C3RuntimeException_wrapIt [8363.1266]
message: "wrapped RuntimeException: Field 'i' does not exist in expression: '[latLongPair, ascending(i), ascending(j)]' for type 'SurfaceHindcastDataSeries'."
JSON: {"spec": {"type": "EvalMetricsSpec", "start": "2021-09-01T00:00:00", "end": "2021-09-30T00:00:00", "timeZone": "NONE", "interval": "DAY", "cache": false, "ids": ["GOMu0.04_1-0"], "expressions": ["TestAverageWaterU"], "continueOnError": false}}

In [70]:
a = evalMetricsResult.result["GOMu0.04_1-1"]["TestAverageWaterU"].m_data

In [71]:
a[1]

TypeError: 'NoneType' object is not subscriptable

In [32]:
df = c3.EvalMetricsResult.toPandas(result=evalMetricsResult)

,TestAverageWaterU
GOMu0.04_1-0_2021-09-01T00:00:00,0.0
GOMu0.04_1-0_2021-09-02T00:00:00,0.0
GOMu0.04_1-0_2021-09-03T00:00:00,0.0
GOMu0.04_1-0_2021-09-04T00:00:00,0.0
GOMu0.04_1-0_2021-09-05T00:00:00,0.0
...,...
GOMu0.04_9-9_2021-09-25T00:00:00,0.0
GOMu0.04_9-9_2021-09-26T00:00:00,0.0
GOMu0.04_9-9_2021-09-27T00:00:00,0.0
GOMu0.04_9-9_2021-09-28T00:00:00,0.0


In [17]:
# convert EvalMetricsResult to Pandas DataFrame
df = c3.EvalMetricsResult.toPandas(result=evalMetricsResult)
df

,AverageWaterU
GOMu0.04_1-1_2021-09-01T00:00:00,0.0
GOMu0.04_1-1_2021-09-01T01:00:00,0.0
GOMu0.04_1-1_2021-09-01T02:00:00,0.0
GOMu0.04_1-1_2021-09-01T03:00:00,0.0
GOMu0.04_1-1_2021-09-01T04:00:00,0.0
...,...
GOMu0.04_1-1_2021-09-29T19:00:00,0.0
GOMu0.04_1-1_2021-09-29T20:00:00,0.0
GOMu0.04_1-1_2021-09-29T21:00:00,0.0
GOMu0.04_1-1_2021-09-29T22:00:00,0.0


In [44]:
import numpy as np

In [40]:
objs_list = c3.SurfaceHindcastDataSeries.fetch(
    spec={"include": "data, data.water_u",
          "filter": "latLongPair.lat > 18 && latLongPair.lat < 19 && latLongPair.lon > -98 && latLongPair.lon < -97"
         }
    ).objs

data_list = [obj.data for obj in objs_list]

In [52]:
water_u_arr = [np.array([obj.water_u for obj in arr]) for arr in data_list]

In [ ]:
# Next Steps:
# 1) Understand the math functions and why it does what it does?
# 2) How can we do lat-lon filtering?
# 3) Implement full pipeline for a specific use-case and compare time-performance